In [ ]:
!pip install transformers
!pip install torchvision

In [20]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
from torch.utils import checkpoint
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class HumanPrediction:
    def __init__(self, model_path):
        self.tokenizer = AutoTokenizer.from_pretrained(
            "bert-base-uncased",
            do_lower_case=True
        )
        self.model = torch.load(model_path)

    def predict(self, text):
        encoded = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=False,
            max_length=512,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = torch.LongTensor([encoded['input_ids']])
        mask = torch.LongTensor([encoded['attention_mask']])
        return float(self.model(ids.to(device), mask.to(device))[0].cpu().detach().numpy())

In [27]:
class CustomModel(torch.nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.distill_bert = AutoModel.from_pretrained(
            "bert-base-uncased"
        )
        
        self.top1 = torch.nn.Linear(768, 64)
        self.top2 = torch.nn.Linear(64, 1)  # classify the data into N laws described in the training dataset

        self.dropout1 = torch.nn.Dropout(p=0.3)
        self.sigm = torch.nn.Sigmoid()

    def forward(self, ids, mask):
        x = self.distill_bert(ids, mask)[0][:, 0, :]
        # select the first token only as it is trained for classifier
        x = torch.nn.functional.relu(self.top1(x))
        x = self.dropout1(x)
        x = self.top2(x)
        return self.sigm(x)

In [28]:
model = HumanPrediction('/home/jupyter/makethon/TUM-Makeathon-22/model_human.bin')

In [29]:
model.predict("""The AI-based monitoring of transactions in real time can support financial institutions in combating money laundering and payment providers: in the inside in the detection of fraud. Real-time payments generated data are fed into the AI ​​system, which then identifies suspicious transactions, the processing of which stops and the transaction for further review by human compliance officer is marked. The fraud detection system is based on AI algorithms that identify patterns and identify connections within the data, which are then clustered and classified. Over time, the system is getting used to the data and the accuracy of the detection increases.""")

0.1841166913509369